**Clustering - Same dataset but using Gaussian Mixture Method**

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
import pyspark.sql.functions as f
import os, sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import GaussianMixture
from pyspark.ml import Pipeline

In [3]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [4]:
from pyspark.ml import Pipeline

In [5]:
from pyspark.ml.feature import StandardScaler

In [6]:
conf = SparkConf()
conf.set("spark.app.name","clustering")
conf.set("spark.master","local[*]")
conf.set("spark.driver.memory","8g")

In [7]:
spark = SparkSession.builder\
                    .config(conf=conf)\
                    .getOrCreate()

**read in the data:**

In [8]:
data_without_header = spark.read\
                            .format("csv")\
                            .option("header",False)\
                            .option("inferSchema",True)\
                            .load(r"C:\Users\blais\Documents\ML\data\kddcup\kddcup.data.corrected")

In [9]:
column_names =  ["duration", "protocol_type", "service", "flag",
                "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent",
                "hot", "num_failed_logins", "logged_in", "num_compromised",
                "root_shell", "su_attempted", "num_root", "num_file_creations",
                "num_shells", "num_access_files", "num_outbound_cmds",
                "is_host_login", "is_guest_login", "count", "srv_count",
                "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate",
                "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
                "dst_host_count", "dst_host_srv_count",
                "dst_host_same_srv_rate", "dst_host_diff_srv_rate",
                "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate",
                "dst_host_serror_rate", "dst_host_srv_serror_rate",
                "dst_host_rerror_rate", "dst_host_srv_rerror_rate",
                "label"]

In [10]:
data = data_without_header.toDF(*column_names)

**Preprocess the data - onehot encode the categorical variables:**
- categorical variables are protocol_type, service, flag

In [11]:
def onehot_pipeline(input_col):
    string_indexer = StringIndexer().setInputCol(input_col).setOutputCol(input_col+'_indexed')
    ohe = OneHotEncoder().setInputCol(input_col+'_indexed').setOutputCol(input_col+ '_encoded')
    enc_pipeline = Pipeline().setStages([string_indexer,ohe])
    return enc_pipeline

In [12]:
data2 = data

In [13]:
for col in ['protocol_type','service','flag']:
    onehot_encoder = onehot_pipeline(col)
    onehot_encoder = onehot_encoder.fit(data2)
    data2 = onehot_encoder.transform(data2) 

Done one-hot encoding.

last bit - standardize other inputs and then feed into model:

In [14]:
list(filter(lambda x: x.startswith('protocol_type') or x.startswith('service') or x.startswith('flag'), data2.columns)) + ['label']

['protocol_type',
 'service',
 'flag',
 'protocol_type_indexed',
 'protocol_type_encoded',
 'service_indexed',
 'service_encoded',
 'flag_indexed',
 'flag_encoded',
 'label']

In [15]:
excluded_columns = list(filter(lambda x: x.startswith('protocol_type') or x.startswith('service') or x.startswith('flag'), data2.columns)) + ['label']

In [16]:
# these should be standardized
includedColumns = list(filter(lambda x: x not in excluded_columns, data2.columns))

In [17]:
assembler1 = VectorAssembler().setInputCols(includedColumns)\
                              .setOutputCol("featureVector1")
scaler = StandardScaler().setInputCol("featureVector1").setOutputCol("standardized_inputs").setWithStd(True).setWithMean(True)
assembler2 = VectorAssembler().setInputCols(["standardized_inputs","protocol_type_encoded","service_encoded","flag_encoded"])\
                              .setOutputCol("featureVector")
gmm = GaussianMixture(featuresCol="featureVector",predictionCol='clusters',probabilityCol='probability',tol=0.01,maxIter=10,
                      seed=42,k=2)

In [18]:
pipeline = Pipeline(stages=[assembler1,scaler,assembler2,gmm])

In [19]:
pipeline_model = pipeline.fit(data2)

get some stats about the model:

In [20]:
gaussian_model = pipeline_model.stages[-1]

these are the arbitrary weights of each gaussian distribution that generated each of the clusters:

In [23]:
gaussian_model.weights

[0.5, 0.5]

Each cluster, modelled by a gaussian distribution has a mean and covariance matrix:

In [24]:
gaussian_model.gaussians[0].mean

DenseVector([0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.5785, 0.3819, 0.574, 0.2247, 0.1272, 0.0197, 0.0148, 0.0118, 0.0083, 0.0033, 0.0014, 0.0011, 0.0011, 0.0009, 0.0008, 0.0007, 0.0004, 0.0003, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7644, 0.1776, 0.0549, 0.0017, 0.0011, 0.0002, 0.0001, 0.0, 0.0, 0.0])

the generated mean is the same dimension as the data's input features and can hence show or imply centres in the features that are important.

Means represent the centers of the clusters in the feature space. 

obviously the mean is the same dimension as the features:

In [30]:
gaussian_model.gaussians[0].mean.shape

(119,)

In [33]:
gaussian_model.gaussians[1].cov

DenseMatrix(119, 119, [1.0, 0.0412, 0.0204, -0.0002, -0.001, 0.0038, 0.0044, 0.0074, ..., -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0], 0)

Shape of the covariance matrix is features by features => 
- Covariance matrix describes the spread and relationships between pairs of features for the k-th gaussian component
- Diagonal elements are variances of each feature indicating how spread out the data is along that feature for the gaussian
- Off-Diagonal elements - represent the covariance between different features showing how they vary together
- Shape dxd

Get log likelihood:

In [36]:
gaussian_model.summary.logLikelihood

1248281190.3349934

Soft and hard clustering:

In [26]:
data2 = pipeline_model.transform(data2)

In [27]:
data2.select("clusters","probability").show()

+--------+-----------+
|clusters|probability|
+--------+-----------+
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
|       0|  [0.5,0.5]|
+--------+-----------+
only showing top 20 rows



**Anomaly Detection:**
- One way of doing anomaly detection with the GMM is inspect the soft clustering - probabilities column and if all probabilities for that point belonging to a certain cluster are all lower than some threshold - i.e. it has no dominants then it is an anomaly. 

**Selecting the number of clusters:**
- Evaluate and compare AIC and BIC values


In [34]:
def get_num_parameters(k, n_features):
    # Mean: k*d parameters
    # Covariance: k*(d*(d+1)/2) parameters (symmetric matrix)
    # Weights: k-1 free parameters (weights sum up to 1)
    return k * (n_features + (n_features * (n_features+1))/2) + (k-1)

In [37]:
gaussian_model.summary.logLikelihood

1248281190.3349934

In [38]:
gaussian_model.getK()

2

In [50]:
import numpy as np

In [51]:
# function to compute AIC and BIC:
def compute_aic_bic(model, data_count, n_features):
    log_likelihood = model.summary.logLikelihood
    p = get_num_parameters(model.getK(),n_features)
    n = data_count
    aic = -2*log_likelihood + 2*p
    bic = -2*log_likelihood + p*np.log(n)
    return aic, bic

Compute d - num of features

In [42]:
len(data2.select("featureVector").first()[0])

119

In [46]:
n_features = len(data2.select("featureVector").first()[0])

In [47]:
data_count = data.count()

In [45]:
data2.select("featureVector").first()[0]

SparseVector(119, {0: -0.0668, 1: -0.0017, 2: 0.0682, 3: -0.0024, 4: -0.0151, 5: -0.0011, 6: -0.0265, 7: -0.0044, 8: 2.4428, 9: -0.0021, 10: -0.0083, 11: -0.0045, 12: -0.0033, 13: -0.0096, 14: -0.0085, 15: -0.0288, 17: -0.0006, 18: -0.0289, 19: -1.5754, 20: -1.1962, 21: -0.466, 22: -0.4658, 23: -0.2483, 24: -0.2481, 25: 0.5397, 26: -0.2561, 27: -0.2011, 28: -3.6391, 29: -1.7865, 30: -1.833, 31: -0.2829, 32: -1.2579, 33: -0.1567, 34: -0.4664, 35: -0.4655, 36: -0.2508, 37: -0.2496, 39: 1.0, 42: 1.0, 109: 1.0})

In [52]:
aic,bic = compute_aic_bic(model=gaussian_model, data_count=data_count, n_features=n_features)

In [53]:
aic

-2496533342.6699867

In [54]:
bic

-2496338723.81604

Plot AIC and BIC against K. 
Choose elbow point.

Other points to note:

its possible to relate the original categories to the indexed integers with the string indexer - using the indexer.labels => returns a tuple of index and category. 